In [1]:
# imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [ ]:
# load data
dataset_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)), 'dataset')
dataset_file = os.path.join(dataset_dir, 'iris.data')